<a href="https://colab.research.google.com/github/Jakeriqu/Microsoft-E5-Auto-Renewal/blob/main/Netflix_Cookie_Checker_(Official).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title **Bước 1: Xây dựng môi trường** { vertical-output: true}
%%capture
!apt-get install -y wget bzip2 libxtst6 libgtk-3-0 libx11-xcb-dev libdbus-glib-1-2 libxt6 libpci-dev xvfb unrar && rm -rf /var/lib/apt/lists/*
!pip install pyvirtualdisplay selenium tqdm requests_toolbelt pyarmor
from pyvirtualdisplay import Display
display = Display(visible=0, size=(800, 600))
display.start()
from google.colab import output
!git clone https://gitea.com/phathua/NetflixCookieChecker.git
%cd NetflixCookieChecker
!mkdir cookies working_cookies
output.clear()
print("Đã thực thi thành công")

In [ ]:
# @title [Thử nghiệm] Nhập code để đăng nhập TV (tùy chọn)
!python3 tv.py

**Lưu ý:** Sau khi chạy bước 1 thì mới dùng được tool nhập code TV

In [ ]:
# @title **Bước 2: Upload file nén cookie (định dạng zip hoặc rar)** { vertical-output: true }
import os
import shutil
import zipfile
from ipywidgets import Button, Text, HBox, Label
from IPython.display import display
from google.colab import files

folder = 'cookies'

# Xóa tất cả các file và thư mục con trong thư mục 'folder'
shutil.rmtree(folder)
os.makedirs(folder)

print("Upload file nén Cookie")

# Tải lên file
uploaded = files.upload()

# Tạo trường nhập mật khẩu
password_field = Text(description='')
optional_label = Label(value="Mật khẩu (có thể bỏ trống):")
def on_button_clicked(b):
    password = password_field.value
    for filename in uploaded.keys():
        try:
            # Kiểm tra xem file có phải là file zip hay không
            if filename.endswith('.zip'):
                # Nếu là file zip, giải nén nó
                with zipfile.ZipFile(filename, 'r') as zip_ref:
                    zip_ref.extractall(folder, pwd=bytes(password,'utf-8') if password else None)
            elif filename.endswith('.rar'):
                # Nếu là file rar, giải nén nó
                if password:
                    !unrar x -p"$password" -y "$filename" "$folder"
                else:
                    !unrar x -y "$filename" "$folder"
            output.clear()
            print("Đã thực thi thành công")
        except RuntimeError:
            output.clear()
            print("Sai mật khẩu")
            print("Thực thi thất bại")
        finally:
            os.remove(filename)

    # Di chuyển tất cả các file từ thư mục con vào thư mục 'folder'
    for root, dirs, files in os.walk(folder):
        for file in files:
            if root != folder:
                src_file = os.path.join(root, file)
                dst_file = os.path.join(folder, file)
                os.replace(src_file, dst_file)

button = Button(description="Giải nén")
button.on_click(on_button_clicked)

# Hiển thị trường nhập mật khẩu và nút
display(HBox([optional_label, password_field, button]))

In [ ]:
# @title **Bước 3: Tự phát hiện chuyển đổi file Netscape sang dạng JSON** { vertical-output: true }
!python3 cookie_converter.py
output.clear()
print("Đã thực thi thành công")

In [ ]:
# @title **Bước 4: Lọc loại tài khoản**
import ipywidgets as widgets
from IPython.display import display
import json
import os
import shutil
import subprocess

# Xóa tất cả các file và thư mục con trong thư mục 'folder'
if os.path.exists('filtered_cookies'):
    shutil.rmtree('filtered_cookies')
os.makedirs('filtered_cookies')

# Đếm số lượng tài khoản cho mỗi loại
account_types = ["Basic", "Standart", "Premium"]
count = {account_type: 0 for account_type in account_types}
files = os.listdir("json_cookies")
for filename in files:
    filepath = os.path.join("json_cookies", filename)
    if os.path.isfile(filepath):
        for account_type in account_types:
            if filename.startswith(f"[{account_type} -"):
                count[account_type] += 1

# Tạo các checkbox
checkbox_basic = widgets.Checkbox(value=True, description=f'Basic ({count["Basic"]})')
checkbox_standart = widgets.Checkbox(value=True, description=f'Standart ({count["Standart"]})')
checkbox_premium = widgets.Checkbox(value=True, description=f'Premium ({count["Premium"]})')

# Hiển thị các checkbox
display(checkbox_basic)
display(checkbox_standart)
display(checkbox_premium)

# Tạo nút submit
button = widgets.Button(description="Thực thi")

# Hàm xử lý khi nút submit được nhấn
def on_button_clicked(b):
    account_types = []
    if checkbox_basic.value:
        account_types.append("Basic")
    if checkbox_standart.value:
        account_types.append("Standart")
    if checkbox_premium.value:
        account_types.append("Premium")

    for filename in files:
        filepath = os.path.join("json_cookies", filename)
        if os.path.isfile(filepath) and any(filename.startswith(f"[{account_type} -") for account_type in account_types):
            shutil.copy(filepath, 'filtered_cookies')
    output.clear()

button.on_click(on_button_clicked)
display(button)

In [ ]:
# @title **Bước 5: Check cookie tự động và tạo link tải về**
!python3 mains.py